# Deriving Coupled Cluster Equations using Sympy

In [1]:
from sympy.physics.secondquant import (AntiSymmetricTensor, wicks,
        F, Fd, NO, evaluate_deltas, substitute_dummies, Commutator,
        simplify_index_permutations, PermutationOperator)
from sympy import (
    symbols, Rational, latex, Dummy
)
import re
from tchau_spin import *
Tensor.rhf = True

In [2]:
pretty_dummies_dict = {
    'above': 'cdefgh',
    'below': 'klmno',
    'general': 'pqrstu'
}

## Define indexes

In [3]:
i = symbols('i', below_fermi=True, cls=Dummy)
a = symbols('a', above_fermi=True, cls=Dummy)
j = symbols('j', below_fermi=True, cls=Dummy)
b = symbols('b', above_fermi=True, cls=Dummy)
p, q, r, s = symbols('p,q,r,s', cls=Dummy)

## Define Hamiltonian

In [4]:
fock = AntiSymmetricTensor('f', (p,), (q,))
pr = NO(Fd(p)*F(q))
V = AntiSymmetricTensor('v',(p,q),(r,s))
pqsr = NO(Fd(p)*Fd(q)*F(s)*F(r))
H = fock*pr + Rational(1,4)*V*pqsr
H

AntiSymmetricTensor(f, (_p,), (_q,))*NO(CreateFermion(_p)*AnnihilateFermion(_q)) - AntiSymmetricTensor(v, (_p, _q), (_r, _s))*NO(CreateFermion(_p)*CreateFermion(_q)*AnnihilateFermion(_r)*AnnihilateFermion(_s))/4

## Define Cluster Operator

In [5]:
def get_T():
    i, j = symbols('i,j', below_fermi=True, cls=Dummy)
    a, b = symbols('a,b', above_fermi=True, cls=Dummy)
    t_ai = AntiSymmetricTensor('t', (a,), (i,))*NO(Fd(a)*F(i))
    t_abij = Rational(1,4)*AntiSymmetricTensor('t', (a,b), (i,j))*NO(Fd(a)*Fd(b)*F(j)*F(i))
    return t_ai + t_abij

## Perform the Hausdorff expansion

In [6]:
C = Commutator
T = get_T()
print("commutator 1...")
comm1 = wicks(C(H, T))
comm1 = evaluate_deltas(comm1)
comm1 = substitute_dummies(comm1)

commutator 1...


In [7]:
T = get_T()
print("commutator 2...")
comm2 = wicks(C(comm1, T))
comm2 = evaluate_deltas(comm2)
comm2 = substitute_dummies(comm2)

commutator 2...


In [8]:
T = get_T()
print("commutator 3...")
comm3 = wicks(C(comm2, T))
comm3 = evaluate_deltas(comm3)
comm3 = substitute_dummies(comm3)

commutator 3...


In [9]:
T = get_T()
print("commutator 4...")
comm4 = wicks(C(comm3, T))
comm4 = evaluate_deltas(comm4)
comm4 = substitute_dummies(comm4)

commutator 4...


## Construct the Similarty Transformed Hamiltonian

In [10]:
eq = H + comm1 + comm2/2 + comm3/6 + comm4/24
eq = eq.expand()
eq = evaluate_deltas(eq)
eq = substitute_dummies(eq, new_indices=True,
        pretty_indices=pretty_dummies_dict)

## Get energy expression

In [11]:
i, j, k, l = symbols('i,j,k,l', below_fermi=True)
a, b, c, d = symbols('a,b,c,d', above_fermi=True)
print()
print("CC Energy:")
energy = wicks(eq, simplify_dummies=True,
        keep_only_fully_contracted=True)
latex(energy)


CC Energy:


'f^{k}_{c} t^{c}_{k} - \\frac{t^{c}_{l} t^{d}_{k} v^{kl}_{cd}}{2} + \\frac{t^{cd}_{kl} v^{kl}_{cd}}{4}'

## Get amplitude Equations

In [12]:
eqT1 = wicks(NO(Fd(i)*F(a))*eq, simplify_kronecker_deltas=True, keep_only_fully_contracted=True)
eqT1 = substitute_dummies(eqT1)
latex(eqT1)

'- f^{k}_{c} t^{c}_{i} t^{a}_{k} + f^{k}_{c} t^{ac}_{ik} - f^{k}_{i} t^{a}_{k} + f^{a}_{c} t^{c}_{i} + f^{a}_{i} - t^{c}_{k} t^{d}_{i} t^{a}_{l} v^{kl}_{cd} - t^{c}_{k} t^{d}_{i} v^{ak}_{cd} + t^{c}_{k} t^{a}_{l} v^{kl}_{ic} + t^{c}_{k} t^{ad}_{il} v^{kl}_{cd} + t^{c}_{k} v^{ak}_{ic} - \\frac{t^{c}_{i} t^{ad}_{kl} v^{kl}_{cd}}{2} + \\frac{t^{a}_{l} t^{cd}_{ik} v^{kl}_{cd}}{2} + \\frac{t^{cd}_{ik} v^{ak}_{cd}}{2} - \\frac{t^{ac}_{kl} v^{kl}_{ic}}{2}'

In [13]:
eqT2 = wicks(NO(Fd(i)*Fd(j)*F(b)*F(a))*eq, simplify_dummies=True, 
             keep_only_fully_contracted=True, simplify_kronecker_deltas=True)
x = latex(eqT2)

In [16]:
latexT1 = latex(eqT1)
print(latexT1)

- f^{k}_{c} t^{c}_{i} t^{a}_{k} + f^{k}_{c} t^{ac}_{ik} - f^{k}_{i} t^{a}_{k} + f^{a}_{c} t^{c}_{i} + f^{a}_{i} - t^{c}_{k} t^{d}_{i} t^{a}_{l} v^{kl}_{cd} - t^{c}_{k} t^{d}_{i} v^{ak}_{cd} + t^{c}_{k} t^{a}_{l} v^{kl}_{ic} + t^{c}_{k} t^{ad}_{il} v^{kl}_{cd} + t^{c}_{k} v^{ak}_{ic} - \frac{t^{c}_{i} t^{ad}_{kl} v^{kl}_{cd}}{2} + \frac{t^{a}_{l} t^{cd}_{ik} v^{kl}_{cd}}{2} + \frac{t^{cd}_{ik} v^{ak}_{cd}}{2} - \frac{t^{ac}_{kl} v^{kl}_{ic}}{2}


In [18]:
i,a = Index.new('ia', 'aa', 'hp')
k,l,c,d = Index.new('klcd', 'xxxx', 'hhpp')
index_key = {'i':i,
             'a':a,
             'k':k,
             'l':l,
             'c':c,
             'd':d
}

platex(eqfromlatex(latexT1, index_key))

<IPython.core.display.Math object>